## Подключение библиотек

In [1]:
#IMPORT LIBS 

#numpy
import numpy as np

#torch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim

#parallel
import joblib
from joblib import Parallel, delayed

#display
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import utils
import unet
import net_process
from itertools import product

import cv2

import os
import copy
import glob
from skimage.io import imread

In [2]:
device

device(type='cuda')

## Загрузчик картинок и генератор датасета

Внутри уже происходит расчет масок и аугментация

In [3]:
images = utils.simple_download('pics')

In [4]:
a = images[7]
b = images[8]
c = images[5]
d = images[6]

tr = utils.Compose(utils.ToTensor(), utils.Normalize())

train_dataset = utils.GenerateDataset([a, c], transforms=tr)
train_data_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# UNET

Обучение сети: **Для тестирования выберите пункт "загрузка сети"**

In [ ]:
%%time

net = unet.UNet().to(device)
net = nn.DataParallel(net)
opt = optim.Adam(net.parameters(), lr=1e-04)

criterion = nn.BCEWithLogitsLoss()
loss, accuracy = net_process.train(net, train_data_loader, opt, device,
                                   criterion, num_epochs=6, verbose_num_iters=3)
print(f'Loss: {loss:.3f}, Accuracy: {accuracy:.3f}')

In [ ]:
torch.save(net, 'net')

**Загрузка сети**

In [5]:
model = torch.load('net')
model.eval()

DataParallel(
  (module): UNet(
    (encoder): VGG13Encoder(
      (blocks): ModuleList(
        (0): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): ReLU(inplace=True)
        )
        (1): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): ReLU(inplace=True)
        )
        (2): Sequential(
          (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): ReLU(inplace=True)
        )
        (3): Sequential(
          (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

Результаты сегментации:

In [6]:
images = [imread(file) for file in glob.glob('Data/*')]
name = glob.glob('Data/*')

In [7]:
names = []
for n in name:
    names.append(n[5:-4])

In [8]:
names

['Pict_3_1',
 'Pict_2_2',
 'Pict_4_1',
 'Pict_2_5',
 'Pict_2_6',
 'Pict_4_3',
 'Pict_4_2',
 'Pict_3_4',
 'Pict_2_1',
 'Pict_2_4',
 'Pict_1_2',
 'Pict_3_2',
 'Pict_1_3',
 'Pict_1_4',
 'Pict_4_4',
 'Pict_2_3',
 'Pict_1_1',
 'Pict_3_3']

Результаты классификации

In [9]:
%mkdir results

mkdir: cannot create directory ‘results’: File exists


In [ ]:
net_process.print_classifications(model, images, names, tr, 8, 55, 0)